In [269]:
import os
import pandas as pd
import geo_utils as geou
import vector_utils as vu
import raster_utils as ru
import plot_utils as pu

In [270]:
import ee

ee.Authenticate()
ee.Initialize()

In [288]:
## County, State abbreviation
county_state = "Santa Barbara, CA" 

## acs hierarchy region
hierarch_region="tract"

## remove water bodies like lakes from shape
rm_water = False

## output directory 
out_dir=r"C:\Users\laure\projects\geo-tlbx-main"

## output driver type (see driver_extension_dict below for options)
out_driver = "ESRI Shapefile"

## ACS survey year
acs_yr = 2022

In [289]:
driver_extension_dict = {
    "ESRI Shapefile" : ".shp",
    "OpenFileGDB" : ".gdb",
    "GPKG" : ".gpkg",
    "GeoJSON" : ".geojson"
}

acs_name_var_dict = { ## {ACS variable name: [ACS variable code to search, column name to save as]}
    'median_household_income': ['B19049_001E', 'mhi'],
     'total_population': ['B01003_001E', 'total_pop'],
     'median_age': ['B01002_001E', 'med_age']
}

## Census 

https://walker-data.com/umich-workshop-2023/python/#76



In [ ]:
import pandas as pd
from functools import reduce
from pygris.utils import erase_water

county_name, state_abr= county_state.split(", ")

acs_vars = []
for acs_name in acs_name_var_dict.keys():
    ## set ACS variable ID from acs_name_var_dict
    acs_var_id = acs_name_var_dict[acs_name][0]
    ## download census shape and vairbale 
    acs_var_gdf = geou.get_census_var_gdf(acs_var_id, acs_yr, hierarch_region, county_name, state_abr)
    ## column limit is 9 for ESRI shapefiles 
    acs_var_gdf.rename(columns={acs_var_id:acs_name_var_dict[acs_name][1]}, inplace=True) 
    ## AGOL features need to be in web mercator (EPSG 3857)
    acs_webmerc = acs_var_gdf.to_crs(3857)
    ## append acs variable gdf to list 
    acs_vars.append(acs_webmerc)

## combine 
acs_var_gdfs = reduce(lambda x, y: pd.merge(x, y, on=acs_webmerc.columns.to_list()[:-1]), acs_vars)

## option to erase water 
if rm_water == True:
    acs_var_gdfs = erase_water(acs_var_gdfs, area_threshold = 0.9)
    
## export 
acs_var_gdfs.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_"+hierarch_region+"_"+driver_extension_dict[out_driver]), driver=out_driver)

## for vis (last key/var in list)
acs_var_gdfs.explore(column = acs_name_var_dict[acs_name][1])


## Open Street Map POIs for running route: 

In [279]:
single_gdf = vu.largest_multipolypart(acs_webmerc.to_crs(4326).dissolve())

single_geom = single_gdf.iloc[0].geometry

## not using currently 
coord_list = [i for i in single_geom.boundary.coords]

## flip XY for 4326 -> 3857: Earth Engine Webmap center 
AOI_center = (single_geom.centroid.y, single_geom.centroid.x)

## OSM bbox
min_lon, min_lat, max_lon, max_lat = tuple([i for i in vu.largest_multipolypart(acs_webmerc).to_crs(4326).total_bounds])
osm_bbox=(min_lat, min_lon, max_lat, max_lon) ## min_lon + 1 to make query end more east and not be in the ocean 

## hard code sb (issues finding OSM & DEM points in the ocean) *********
## osm_bbox = (34.412986361497566, -120.58261366193953, 34.829092385785955, -119.54844924629602)

In [280]:
POI_dict = {
    "other_water":["drinking_water", "pharmacy", "school"]
    , "all_others": ["toilets"]
            }

for poi_type in POI_dict.keys():
    pois = []
    for poi_name in POI_dict[poi_type]:
        poi = geou.osm_poi(osm_bbox, poi_name)
        if len(poi) > 1:
            poi['amenity'] = [poi_name for i,v in poi.iterrows()]
            pois.append(poi)
    pois_merge = pd.concat(pois)
    pois_merge.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_"+poi_type+"_OSM_POIs"+driver_extension_dict[out_driver]), driver=out_driver)

## find water crossings as POIs for water sources 

In [ ]:
## download OSM streams
waterways_gdf = geou.osm_lines(bbox = osm_bbox, waytype = "waterway").to_crs(3857)
streams = waterways_gdf[waterways_gdf['type'] == 'stream']
streams.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_OSM_streams"+driver_extension_dict[out_driver]), driver=out_driver)

In [282]:
## download OSM roads
roads = geou.osm_lines(bbox = osm_bbox, waytype = "highway").to_crs(3857)
## save OSM roads to file
roads.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_osm_roads.gpkg"), driver="GPKG")
## print road types that will be used to subset trails 
print(list(roads['type'].unique()))
## look at shape in QGIS against satellite basemap -- go to my trails, see what they're classified as   
trailtypes = ['track', 'path', 'unclassified', 'footway', 'pedestrian', 'bridleway']
## subset trails
trails = roads[roads['type'].isin(trailtypes)]
## save trails
trails.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_OSM_trails"+driver_extension_dict[out_driver]), driver=out_driver)

['cycleway', 'path', 'unclassified', 'tertiary', 'residential', 'track', 'service', 'trunk_link', 'footway', 'motorway_link', 'living_street', 'primary_link', 'secondary_link', 'primary', 'secondary', 'disused', 'trunk', 'motorway', 'pedestrian', 'tertiary_link', 'bridleway', 'construction', 'steps', 'corridor', 'raceway', 'proposed', 'traffic_island', 'elevator', 'platform']


In [283]:
## find intersecting stream and trail lines 
xing_pts, xing_lines = vu.line_intersections(geom1 = trails.dissolve().to_crs(4326).geometry, 
                                          geom2 = streams.dissolve().to_crs(4326).geometry)
xing_pts.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_OSM_h2oXing_pts"+driver_extension_dict[out_driver]), driver=out_driver)
xing_lines.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_OSM_h2oXing_lines"+driver_extension_dict[out_driver]), driver=out_driver)

### find OSM roads (lots of trails) missing in Census roads dataset

In [ ]:
## get Census - TIGER primary roads 
acs_roads = geou.get_roads_by_county(county = county_name, state = state_abr).to_crs(3857) 
## save to file
acs_roads.to_file(os.path.join(out_dir, county_name.replace(" ", "")+"_ACS_roads"+driver_extension_dict[out_driver]), driver=out_driver)
acs_roads.explore('RTTYP')

In [285]:
## find intersecting stream and trail lines 
xing_pts, xing_lines = vu.line_intersections(geom1 = roads.dissolve().to_crs(4326).geometry, 
                                          geom2 = acs_roads.dissolve().to_crs(4326).geometry)

## US 3DEP - Elevation


In [296]:
import py3dep
import rioxarray

## NAD83 horizontal datum, NAVD88 vertical datum
dem = py3dep.static_3dep_dem(geometry = vu.largest_multipolypart(acs_webmerc.dissolve()).geometry.iloc[0], crs = 3857, resolution = 10)
dem.rio.to_raster(os.path.join(out_dir, county_name.replace(" ", "")+"_DEM.tif"))

## GEE - NLCD (Landcover)

In [297]:
## create aoi object for EE (in 4326)
aoi = ee.Geometry.Rectangle([single_gdf.bounds.minx[0], single_gdf.bounds.miny[0], single_gdf.bounds.maxx[0], single_gdf.bounds.maxy[0]])

In [ ]:
import geemap

Map = geemap.Map(center=AOI_center, zoom=4)

nlcd = ee.ImageCollection("USGS/NLCD_RELEASES/2021_REL/NLCD").select("landcover").filterBounds(aoi)
nlcd_colors = nlcd.getInfo()['features'][0]['properties']['landcover_class_palette']
nlcd_values = sorted(nlcd.getInfo()['features'][0]['properties']['landcover_class_values'])
Map.addLayer(nlcd, {'bands':'landcover', 'palette': nlcd_colors, 'min': nlcd_values[0], 'max':nlcd_values[-1]}, "2021 NLCD")
## download image
geemap.download_ee_image_tiles(nlcd.toBands(), geemap.fishnet(aoi, rows=2, cols=2), out_dir, prefix="nlcd_", crs="EPSG:3857", scale=30)
## display map
Map

In [299]:
## mosaic, delete tiles 
nlcd_tiles = [os.path.join(out_dir, i) for i in os.listdir(out_dir) if ("nlcd" in i and i.endswith(".tif"))]
ru.mosaic_rasters(in_rasters = nlcd_tiles, 
                  out_path = os.path.join(out_dir, county_name.replace(" ", "")+"_nlcd.tif"))
for i in nlcd_tiles:
    os.remove(i)

## route optimization
https://openrouteservice.org/example-optimize-pub-crawl-with-ors/

https://www.youtube.com/watch?v=OOCvhc0k1R4



In [ ]:
import openrouteservice as ors
from openrouteservice import distance_matrix

## https://readthedocs.org/projects/openrouteservice-py/downloads/pdf/latest/

POI_coords = list(zip([i.x for i in pois_merge.geometry], [i.y for i in pois_merge.geometry]))
request = {'locations': POI_coords,
           'profile': 'foot-hiking', ## profile options: f [“driving-car”, “driving-hgv”, “foot-walking”, “foot-hiking”, “cycling-regular”, “cycling-road”,”cycling-mountain”, “cycling-electric”,].
           'metrics': ['distance']} ## default duration 
poi_matrix = ors.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(poi_matrix['durations']), len(poi_matrix['durations'][0])))